# FlowNet batch 생성

import (tensorflow, numpy) package <br>
numpy use to make numpy array batch data for training <br>
tensorflow use to make FlowNet structure

In [1]:
import tensorflow as tf
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


dataset : Scene Flow Datasets <br>
model : FlowNet structure <br>

In [2]:
from model import FlowNet
from dataset import Scene_Flow_disparity

Instructions for updating:
Use the retry module or similar alternatives.


Download Scene Flow Datasets <br>
Make data_paths list for loading data images

In [ ]:
dataset = Scene_Flow_disparity()

Check data method in mode = left, groundTruth mode = disparity

In [ ]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)
    else:
        print('no left or png path')

# FlowNet Simple training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make FlowNet simple structure <br>
set training environment(input shape, learning rate, FlowNet structure mode, gpu device set)

In [ ]:
import time
import datetime

start=time.clock()

model = FlowNet(
    img_height = 540,
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('simple')

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
train_left_x = np.array([])
train_left_x_batch = np.array([])
train_right_x = np.array([])
train_right_x_batch = np.array([])
train_y = np.array([])
train_y_batch = np.array([])

iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_,
                                                             (model.model_in_height, model.model_in_width),
                                                             (model.model_out_height, model.model_out_width))
        if train_left_x.size == 0:
            train_left_x = left_image[np.newaxis,:]
            train_right_x = right_image[np.newaxis,:]
            train_y = ground_truth[np.newaxis,:]
        else:
            train_left_x_batch = np.append(train_left_x, left_image[np.newaxis,:], axis = 0)
            train_right_x_batch = np.append(train_right_x, right_image[np.newaxis,:], axis = 0)
            train_y_batch = np.append(train_y, ground_truth[np.newaxis,:], axis = 0)

        iteration += 1

    elif iteration >= 100:
        hist = net.fit([train_left_x_batch, train_right_x_batch], 
                       train_y_batch, 
                       batch_size=8, 
                       epochs=100, 
                       validation_split=0.2)
        if not os.path.exists('./checkpoints/'):
            os.makedirs('./checkpoints/')
        net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')
        train_left_x = np.array([])
        train_left_x_batch = np.array([])
        train_right_x = np.array([])
        train_right_x_batch = np.array([])
        train_y = np.array([])
        train_y_batch = np.array([])
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/flownetSimple_for_depth.hdf5')

# FlowNet Correlation training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make FlowNet simple structure <br>
set training environment(input shape, learning rate, FlowNet structure mode)

In [3]:
import time
import datetime

model = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('correlation')

start=time.clock()

input image resized by (height = 512, width = 960)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
right_image (InputLayer)        (None, 512, 960, 3)  0                                            
__________________________________________________________________________________________________
left_input (InputLayer)         (None, 512, 960, 3)  0                                            
__________________________________________________________________________________________________
right_Conv1 (Conv2D)            (None, 256, 480, 64) 9472        right_image[0][0]                
__________________________________________________________________________________________________
left_Conv1 (Conv2D)             (None, 256, 480, 64) 9472        left_input[0][0]                 
__________________________________________________________

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
train_left_x = np.array([])
train_left_x_batch = np.array([])
train_right_x = np.array([])
train_right_x_batch = np.array([])
train_y = np.array([])
train_y_batch = np.array([])

iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, ground_truth = dataset.data(dir_, 
                                                             (model.model_in_height, model.model_in_width), 
                                                             (model.model_out_height, model.model_out_width))
        
        if train_left_x.size == 0:
            train_left_x = left_image[np.newaxis,:]
            train_right_x = right_image[np.newaxis,:]
            train_y = ground_truth[np.newaxis,:]
        else:
            train_left_x_batch = np.append(train_left_x, left_image[np.newaxis,:], axis = 0)
            train_right_x_batch = np.append(train_right_x, right_image[np.newaxis,:], axis = 0)
            train_y_batch = np.append(train_y, ground_truth[np.newaxis,:], axis = 0)
        
        iteration += 1
        
    elif iteration >= 100:
        hist = net.fit([train_left_x_batch, train_right_x_batch], 
                           train_y_batch, 
                           batch_size=8, 
                           epochs=100, 
                           validation_split=0.2)
        net.save_weights('./checkpoints/flownetCorr_for_depth.hdf5')
        train_left_x = np.array([])
        train_left_x_batch = np.array([])
        train_right_x = np.array([])
        train_right_x_batch = np.array([])
        train_y = np.array([])
        train_y_batch = np.array([])
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/flownetCorr_for_depth.hdf5')

# FlowNet prediction


In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from model import FlowNet
from dataset import Scene_Flow_disparity

In [ ]:
dataset = Scene_Flow_disparity()

In [ ]:
prediction_model = FlowNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

prediction_net = prediction_model.inference('simple')

prediction_net.load_weights('./checkpoints/flownetSimple_for_depth.hdf5')

In [ ]:
import matplotlib.pyplot as plt

left, right, groundtruth = dataset.data('./test_image/img/left/0001.png', 
                              (prediction_model.model_in_height, prediction_model.model_in_width), 
                              (prediction_model.model_out_height, prediction_model.model_out_width))

train_left_x = left[np.newaxis,:]
train_right_x = right[np.newaxis,:]

prediction = prediction_net.predict([train_left_x, train_right_x],
                                    batch_size=1);

plt.imshow(prediction[0,:,:,0])
plt.show()
plt.imshow(groundtruth[:,:,0])
plt.show()
# save .pfm file
# dataset.write_pfm(prediction[0,:,:,0], dir_output+'/%s.pfm' % (image_path.split('/')[-1]))